In [1]:
import gypsy

In [10]:
%%bash
ls ../private-data/

1000-plots.csv
1000-plots.csv_prepped.csv


In [7]:
%%bash
python -m cProfile -s cumtime  "$(which gypsy)" prep ../private-data/1000-plots.csv > profile_prep_1000.txt

INFO     Running prep...
INFO     0% complete
INFO     10% complete
INFO     20% complete
INFO     30% complete
INFO     40% complete
INFO     50% complete
INFO     60% complete
INFO     70% complete
INFO     80% complete
INFO     90% complete


In [9]:
%%bash
cat profile_prep_1000.txt


 GYPSYNonSpatial.densityNonSpatialAw()
 Slow convergence

 asaCompileAgeGivenSpSiHt.computeTreeAge()
 Tree Age Search Routine Terminated; treeAge > 1000
 rowIndex: 0 siSp: Pl treeHt: 26.0 treeSi: 10.7021513718 current treeAge: 1013.50035006

 asaCompileAgeGivenSpSiHt.computeTreeAge()
 Tree Age Search Routine Terminated; treeAge > 1000
 rowIndex: 0 siSp: Pl treeHt: 25.39 treeSi: 10.7537466946 current treeAge: 1001.95421726
         23532691 function calls (22888860 primitive calls) in 163.187 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.021    0.021  163.198  163.198 gypsy:3(<module>)
        1    0.000    0.000  154.965  154.965 core.py:714(__call__)
        1    0.000    0.000  154.965  154.965 core.py:637(main)
        1    0.000    0.000  154.963  154.963 core.py:1021(invoke)
        2    0.000    0.000  154.958   77.479 core.py:884(invoke)
        2    0.000    0.000  154.958   77.479 core.py:497(in

In [ ]:
%%bash
python -m cProfile -s cumtime  "$(which gypsy)" simulate ../private-data/1000-plots.csv_prepped.csv > profile_sim_1000.txt